In [1]:
import cv2
from imutils import paths
import numpy as np
import os
from sklearn.svm import SVC
import joblib
from scipy.cluster.vq import *
from sklearn import preprocessing

In [2]:
# 获取测试数据类别的名称
train_path = r'/Users/zhangxiao/Desktop/数据/训练数据/'
train_names = os.listdir(train_path)
del train_names[0]


In [3]:
# 获取所有图片的路径，保存为一个列表
# 读取每张图片的lable，并保存在一个列表
# 此处实际上每个文件夹里都是相同label的图片，即文件夹名就是里面图片的lable
def imageInfo(train_names):

    image_paths = []
    image_labels = []
    for train_name in train_names:
        dir = os.path.join(train_path, train_name)
        image_path = list(paths.list_images(dir))
        image_paths += image_path
        image_labels += [train_name] * len(image_path)
    return image_paths, image_labels

In [4]:
def create_des(image_paths):

    # 创建SIFT特征提取器
    sift = cv2.xfeatures2d.SIFT_create(700)

    # 特征提取与描述子生成
    des_list = []
    for image_path in image_paths:
        im = cv2.imread(image_path)
        im = cv2.resize(im, (300, 300))
        kpts = sift.detect(im)
        kpts, des = sift.compute(im, kpts)
        des_list.append((image_path, des))
        # print("image file path : ", image_path)

    # 描述子向量
    descriptors = des_list[0][1]
    for image_path, descriptor in des_list[1:]:
        descriptors = np.vstack((descriptors, descriptor))
    print("create finished!")
    return descriptors, des_list

In [5]:
def BoWAndfeaturehist(descriptors, des_list, k):

    # 生成中心词
    center, _ = kmeans(descriptors, k)

    # 生成特征直方图
    im_features = np.zeros((len(image_paths), k), "float32")
    for i in range(len(image_paths)):
        words, distance = vq(des_list[i][1], center)
        for w in words:
            im_features[i][w] += 1
    return im_features, center

In [6]:
# 实现动词词频与出现频率统计,构造TF-IDF向量
k = 100
image_paths, image_labels = imageInfo(train_names)
descriptors, des_list = create_des(image_paths)
im_features, center = BoWAndfeaturehist(descriptors, des_list, k)
nbr_occurences = np.sum((im_features > 0) * 1, axis=0)
idf = np.array(np.log((1.0 * len(image_paths) + 1) / (1.0 * nbr_occurences + 1)), 'float32')
im_features = im_features * idf # tf-idf

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 128 and the array at index 1 has size 1

In [ ]:
im_features.shape